In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def get_point(shape):
    x = randint(50, shape[2] - 50)
    y = randint(50, shape[1] - 50)
    z = randint(20, shape[0] - 20)
    return np.asarray([z, y, x])

def get_patch_from_list(lung_img, coords, window_size = 10):
    shape = lung_img.shape
    output = []
    lung_img = lung_img + 1024
    for i in range(len(coords)):
        patch =   lung_img[coords[i][0] - 18: coords[i][0] + 18,
                           coords[i][1] - 18: coords[i][1] + 18,
                           coords[i][2] - 18: coords[i][2] + 18]               
        output.append(patch)
    
    return output

def create_cls_sample(df_anno,df_pred,img_file,data_path,output_true,output_false):
    coords, trainY = [], []
    mini_df_anno = df_anno[df_anno["file"]==img_file] #get all nodules associate with file
    if mini_df_anno.shape[0]>0: # some files may not have a nodule--skipping those 
        # load the data once
        patient_id = img_file.split('/')[-1][:-4]
        itk_img = SimpleITK.ReadImage(data_path + img_file) 
        img_array = SimpleITK.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
        num_z, height, width = img_array.shape        #heightXwidth constitute the transverse plane
        origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
        spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
        # go through all nodes (why just the biggest?)
        for node_idx, cur_row in mini_df_anno.iterrows(): 
            counter = 0
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            diam = cur_row["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            img_array = normalize(img_array)
            
            new_x = int(v_center[0])
            new_y = int(v_center[1])
            new_z = int(v_center[2])
            for i in range(-5, 5, 3):
                for j in range(-5, 5, 3):
                    for k in range(-2, 3, 2):
                        coords.append([new_z + k, new_y + j, new_x + i])
                        trainY.append(True)

            for i in range(60):            
                coords.append(get_point(img_array.shape))
                trainY.append(False)

            trainX = get_patch_from_list(img_array, coords)
            np.save(output_true + 'traindata_' + str(patient_id) + '_' + str(node_idx) + '_Xtrain.npy',trainX)
            np.save(output_true + 'traindata_' + str(patient_id) + '_' + str(node_idx) + '_Ytrain.npy',np.asarray(trainY, dtype = bool))
            coords, trainY = [], []

In [3]:
csv_path = PATH['annotations_train']
output_true = PATH['cls_train_30_true']
output_false = PATH['cls_train_30_false']
pred_csv_path = PATH['model_train_pred']
data_path = PATH['src_train']
anno_csv_new = pd.read_csv(csv_path + "annotations_all.csv")
pred_csv_new = pd.read_csv(pred_csv_path + "anno_false_final.csv")

In [4]:
pred_csv_new = pred_csv_new[pred_csv_new.index%12 == 0]

In [5]:
patients=load_train(data_path)
anno_csv_new["file"] = anno_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
anno_csv_new = anno_csv_new.dropna()
pred_csv_new["file"] = pred_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
pred_csv_new = pred_csv_new.dropna()

In [6]:
Parallel(n_jobs=-1)(delayed(create_cls_sample)(anno_csv_new,pred_csv_new,patient,data_path,output_true,output_false) for patient in tqdm(sorted(patients)))

100%|██████████| 600/600 [24:45<00:00,  1.70s/it]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,